## Data Ingestion

In [2]:
import os
%pwd

'd:\\Text Sumarizer\\researrch'

In [3]:
os.chdir("../")

In [4]:
%pwd

'd:\\Text Sumarizer'

In [42]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataIngestionConfig:
    root_dir: Path
    source_url: str
    local_data_file: Path
    unzip_dir: Path

In [43]:
from src.text_summarizer.constants import *
from src.text_summarizer.utils.common import read_yaml, create_directories

class ConfigurationManager:
    def __init__(self, config_filepath: str = CONFIG_PATH, params_filepath: str = PARAMS_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        # Create necessary directories
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])  # Ensure directory exists

        return DataIngestionConfig(
            root_dir=config.root_dir,
            source_url=config.source_url,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )

In [44]:
import os
import urllib.request as request
from zipfile import ZipFile
from src.text_summarizer.logging import logging

## Component
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        local_dir = os.path.dirname(self.config.local_data_file)  # Extract parent directory
        os.makedirs(local_dir, exist_ok=True)  # Ensure directory exists

        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_url,
                filename=self.config.local_data_file
            )
            logging.info(f"{filename} downloaded! with following info: \n{headers}")
        else:
            logging.info("File already exists")

    def extract_zip_file(self):
        """
        Extracts the zip file into the data directory.
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)  # Ensure directory exists

        with ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
        logging.info(f"Extracted {self.config.local_data_file} to {unzip_path}")




In [45]:
config=ConfigurationManager()
data_ingestion_config=config.get_data_ingestion_config()
data_ingestion=DataIngestion(config=data_ingestion_config)

data_ingestion.download_file()
data_ingestion.extract_zip_file()


[2025-03-10 02:48:30,908: INFO: common]: yaml file: config\config.yaml loaded successfully
[2025-03-10 02:48:30,912: INFO: common]: yaml file: params.yaml loaded successfully
[2025-03-10 02:48:30,915: INFO: common]: created directory at: Artifacts/data_ingestion
[2025-03-10 02:48:30,918: INFO: common]: created directory at: Artifacts/data_ingestion
[2025-03-10 02:48:41,185: INFO: 687862137]: Artifacts/data_ingestion/data_zip/data.zip downloaded! with following info: 
Connection: close
Content-Length: 7903594
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "dbc016a060da18070593b83afff580c9b300f0b6ea4147a7988433e04df246ca"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 7A56:B1A9F:AA0F6:422B32:67CE0AB9
Accept-Ranges: bytes
Date: Sun, 09 Mar 2025 21:48:32 GMT
Via: 1.1 varnish
X-Served-By: cache